### EDA and Forecasting of Superstore Sales

This project...

#### Library imports and necessary pip installs

In [3]:
!pip install kagglehub

  Using cached kagglehub-0.3.11-py3-none-any.whl.metadata (32 kB)
Using cached kagglehub-0.3.11-py3-none-any.whl (63 kB)



[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import kagglehub
import shutil
import os
import pandas as pd

#### Dataset download

We download the dataset we'll be using from kaggle using the `kagglehub` library

In [34]:
path = kagglehub.dataset_download("rohitsahoo/sales-forecasting")

Since this method by default downloads files to an internal KaggleHub cache location, we'll copy them to our current location. 

In [35]:
destination = os.path.join(os.getcwd(), "./data")
os.makedirs(destination, exist_ok=True)

for filename in os.listdir(path):
    src_file = os.path.join(path, filename)
    dst_file = os.path.join(destination, filename)
    shutil.copy(src_file, dst_file)

We now read the dataset with `pandas` and we can take a look and start exploring the different fields.

In [36]:
df = pd.read_csv(os.path.join(destination, "train.csv"))
df.head(1)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96


We'll get some information about the DataFrame using the `info()` method. The information contains the number of columns, column labels, column data types, memory usage, range index, and the number of cells in each column (non-null values).

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int64  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   float64
dtypes: float

We'll drop some irrelevant or low-impact columns for sales forecasting. These include identifiers and granular personal data that do not contribute significantly to the forecasting task.

We'll also convert the format of `Order Date` add `Ship Date` to standard datetime formats.

In [ ]:
columns_to_drop = [
    'Row ID', 'Order ID', 'Ship Mode', 'Customer ID',
    'Customer Name', 'Product ID', 'Postal Code', 'Country'
    ]

df.drop(columns=columns_to_drop, inplace=True)

df['Order Date'] = pd.to_datetime(df['Order Date'], format="%d/%m/%Y")
df['Ship Date']  = pd.to_datetime(df['Ship Date'], format="%d/%m/%Y")

In [41]:
df.head(5)

,Order Date,Ship Date,Segment,City,State,Region,Category,Sub-Category,Product Name,Sales
0,2017-11-08,2017-11-11,Consumer,Henderson,Kentucky,South,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2017-11-08,2017-11-11,Consumer,Henderson,Kentucky,South,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,2017-06-12,2017-06-16,Corporate,Los Angeles,California,West,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,2016-10-11,2016-10-18,Consumer,Fort Lauderdale,Florida,South,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,2016-10-11,2016-10-18,Consumer,Fort Lauderdale,Florida,South,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
